In [21]:
from pandas import read_excel
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
import pandas as pd

In [2]:
from keywordsdw import GJBResearcherProfile, GJBThesis, Author

In [3]:
engine = create_engine('postgres+psycopg2://postgres@localhost:5434/keywordsdw')

/Users/likit/.virtualenvs/tmob2018/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
session = Session(engine)

In [5]:
df = read_excel('goldenjubilee2018.xlsx')

In [12]:
df.head(1)

,projectid,maincode,secondcode,universitycode,yearcode,advisorid,stdid,title_th,firstname_th,lastname_th,project_status,major,faculty,university,email,thesistitle_thai,thesistitle_eng,title_en,firstname_en,lastname_en
0,PHD41K0177,4,S,NU,41,A,1,น.ส.,ภัคสิริ,สินไชยกิจ,สิ้นสุดโครงการ,วิทยาศาสตร์ชีวภาพ,คณะวิทยาศาสตร์,มหาวิทยาลัยนเรศวร,puksiri1@hotmail.com,ฤทธิ์ต้านอนุมูลอิสระและคุณสมบัติทางชีวภาพของฟล...,Antioxidant and relating biological activities...,NaN,NaN,NaN


In [6]:
df.columns

Index(['projectid', 'maincode', 'secondcode', 'universitycode', 'yearcode',
       'advisorid', 'stdid', 'คำนำหน้าชื่อ', 'ชื่อ', 'นามสกุล', 'สถานะโครงการ',
       'ชื่อสาขา', 'คณะ', 'มหาวิทยาลัย', 'อีเมล์', 'thesistitle_thai',
       'thesistitle_eng', 'PrefixEngStd', 'NameEngStd', 'LastNameEngStd'],
      dtype='object')

In [7]:
df.rename(columns={'คณะ': 'faculty', 'คำนำหน้าชื่อ': 'title_th', 'ชื่อ': 'firstname_th',
                   'PrefixEngStd': 'title_en', 'NameEngStd': 'firstname_en',
                   'LastNameEngStd': 'lastname_en',
                  'นามสกุล': 'lastname_th', 'สถานะโครงการ': 'project_status',
                  'ชื่อสาขา': 'major', 'มหาวิทยาลัย': 'university',
                  'อีเมล์': 'email'}, inplace=True)

In [46]:
for idx,row in df.iterrows():
    gender = 'M' if row['title_th']=='นาย' else 'F'
    project_status = True if row['project_status'] == 'สิ้นสุดโครงการ' else False
    gjb_thesis = GJBThesis(
        finished=project_status,
        title_th=row['thesistitle_thai'],
        title_en=row['thesistitle_eng']
    )
    gjb_researcher = GJBResearcherProfile(
        title_th=row['title_th'],
        title_en=row['title_en'] if not pd.isna(row['title_en']) else None,
        first_name_th=row['firstname_th'],
        last_name_th=row['lastname_th'],
        first_name_en=row['firstname_en'].lower() if not pd.isna(row['firstname_en']) else None,
        last_name_en=row['lastname_en'].lower() if not pd.isna(row['lastname_en']) else None,
        gender=gender,
        email=row['email'],
        major_th=row['major'],
        faculty_th=row['faculty'],
        university_th=row['university'],
    )
    gjb_researcher.theses.append(gjb_thesis)
    session.add(gjb_researcher)
session.commit()

In [42]:
r = row['lastname_en'].lower() if not pd.isna(row['lastname_en']) else None

In [44]:
print(r)

None


In [27]:
pd.isna(row['firstname_en'])

True